In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../src")

In [3]:
from CompoTree import ComponentTree, Radicals, TSVariants, CharLexicon, IDC
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import numpy.linalg as la

In [4]:
kv = KeyedVectors.load("../data/fasttext-zh-sw/fasttext-zh-sw.kv")
kv.init_sims()

In [5]:
ctree = ComponentTree.load()
radicals = Radicals.load()
tsvars = TSVariants.load()
lexicon = CharLexicon.load(topn=5000)

In [6]:
ctree.query("慮", use_flag="shortest", max_depth=2)[0].print_tree()

|(慮): ⿸
..|(虍): ⿸
....|(--): ⿱
......| ⺊
......| ②
....| 七
..|(思): ⿱
....| 田
....| 心


In [7]:
# heartRed_hits = ctree.find("忄", max_depth=3, bmp_only=True)



In [8]:
IDC.vert2

<IdcEnum.vert2: '⿱'>

In [11]:
from CompoTree import CTFounds
heartAll_hits = ctree.find("心", max_depth=3, bmp_only=True)
CTFounds(heartAll_hits)\
    .filter(idc=IDC.vert2.value, pos=1)\
    .filter_with_lexicon(lexicon)\
    .simplified_only()\
    .tolist()

[('恳', [⿱1恳]),
 ('葱', [⿱1葱, ⿱1怱]),
 ('志', [⿱1志]),
 ('总', [⿱1总]),
 ('聪', [⿰1聪, ⿱1总]),
 ('恶', [⿱1恶]),
 ('态', [⿱1态]),
 ('隐', [⿰1隐, ⿱1急]),
 ('瘾', [⿸1瘾, ⿰1隐, ⿱1急]),
 ('稳', [⿰1稳, ⿱1急]),
 ('恋', [⿱1恋]),
 ('懑', [⿱1懑]),
 ('鳃', [⿰1鳃, ⿱1思]),
 ('惫', [⿱1惫, ⿱1思]),
 ('悬', [⿱1悬]),
 ('惩', [⿱1惩]),
 ('怂', [⿱1怂]),
 ('怼', [⿱1怼])]

In [13]:
from CompoTree import CTFounds
heartRed_hits = ctree.find("忄", max_depth=1, bmp_only=True)
CTFounds(heartRed_hits)\
    .filter(idc=IDC.horz2.value, pos=0)\
    .filter_with_lexicon(lexicon)\
    .traditional_only()\
    .tolist()

[('忱', [⿰0忱]),
 ('慄', [⿰0慄]),
 ('悅', [⿰0悅]),
 ('愉', [⿰0愉(GTJ)]),
 ('悸', [⿰0悸]),
 ('悼', [⿰0悼]),
 ('快', [⿰0快]),
 ('憬', [⿰0憬]),
 ('惚', [⿰0惚]),
 ('懨', [⿰0懨]),
 ('惺', [⿰0惺]),
 ('悔', [⿰0悔(GTKV)]),
 ('懈', [⿰0懈]),
 ('慷', [⿰0慷]),
 ('愣', [⿰0愣]),
 ('忤', [⿰0忤]),
 ('惆', [⿰0惆]),
 ('慍', [⿰0慍]),
 ('憶', [⿰0憶]),
 ('悻', [⿰0悻]),
 ('悍', [⿰0悍]),
 ('恃', [⿰0恃]),
 ('愴', [⿰0愴]),
 ('怯', [⿰0怯]),
 ('懵', [⿰0懵]),
 ('怪', [⿰0怪]),
 ('惰', [⿰0惰]),
 ('懼', [⿰0懼]),
 ('惕', [⿰0惕]),
 ('恍', [⿰0恍]),
 ('情', [⿰0情(GTJV)]),
 ('性', [⿰0性]),
 ('忙', [⿰0忙]),
 ('愕', [⿰0愕]),
 ('惶', [⿰0惶]),
 ('慟', [⿰0慟]),
 ('憔', [⿰0憔]),
 ('惱', [⿰0惱]),
 ('悵', [⿰0悵]),
 ('悖', [⿰0悖]),
 ('悌', [⿰0悌]),
 ('慘', [⿰0慘]),
 ('恬', [⿰0恬]),
 ('慚', [⿰0慚]),
 ('慢', [⿰0慢]),
 ('愷', [⿰0愷]),
 ('怕', [⿰0怕]),
 ('怵', [⿰0怵(G)]),
 ('憧', [⿰0憧]),
 ('愧', [⿰0愧]),
 ('悴', [⿰0悴]),
 ('惘', [⿰0惘]),
 ('憾', [⿰0憾]),
 ('懊', [⿰0懊(G)]),
 ('憎', [⿰0憎(GTK)]),
 ('悟', [⿰0悟]),
 ('恰', [⿰0恰]),
 ('惴', [⿰0惴]),
 ('愜', [⿰0愜]),
 ('惦', [⿰0惦]),
 ('忪', [⿰0忪]),
 ('憐', [⿰0憐]),
 ('愾', [⿰0愾]),
 ('懂', [⿰0懂]),
 ('憤', [⿰0憤]

In [60]:
heartRed_char = [x[0] for x in heartRed_hits if x[0] in kv.vocab]
heartAll_char = [x[0] for x in heartAll_hits if x[0] in kv.vocab]

In [63]:
heartRed_char[1]

'愫'

In [85]:
heartRed_trad = []
heartRed_simp = []
heartRed_share = []
for w in heartRed_char:
    if tsvars.is_traditional(w) and tsvars.is_simplified(w) == False:
        heartRed_trad.append(w)
    elif tsvars.is_simplified(w) and tsvars.is_traditional(w) == False:
        heartRed_simp.append(w)
    else:
        heartRed_share.append(w)

In [89]:
heartAll_trad = []
heartAll_simp = []
heartAll_share = []
for w in heartAll_char:
    if tsvars.is_traditional(w) and tsvars.is_simplified(w) == False:
        heartAll_trad.append(w)
    elif tsvars.is_simplified(w) and tsvars.is_traditional(w) == False:
        heartAll_simp.append(w)
    else:
        heartAll_share.append(w)

In [92]:
print(heartAll_simp)

['态', '总', '怂', '虑', '戆', '恋', '慭', '悫', '惩', '闷', '懑', '怼', '悬', '恳', '恶']


In [93]:
print(heartAll_trad)

['憊', '戇', '懟', '態', '悶', '惡', '憑', '懣', '應', '懲', '愨', '懸', '懇', '戀', '憲', '恥']


In [84]:
tsvars.is_simplified('志')

True

In [77]:
heart_sv = kv.vectors_norm[kv.vocab['心'].index,:]

In [109]:
heartRed_trad_cor = [kv.similarity('心', w) for w in heartRed_trad]
heartAll_trad_cor = [kv.similarity('心', w) for w in heartAll_trad]
heartRed_simp_cor = [kv.similarity('心', w) for w in heartRed_simp]
heartAll_simp_cor = [kv.similarity('心', w) for w in heartAll_simp]
heartRed_share_cor = [kv.similarity('心', w) for w in heartRed_share]
heartAll_share_cor = [kv.similarity('心', w) for w in heartAll_share]

In [110]:
np.mean(heartRed_trad_cor), np.mean(heartAll_trad_cor)

(0.946144316613844, 0.9481927658387708)

In [111]:
np.mean(heartRed_simp_cor), np.mean(heartAll_simp_cor)

(0.9502067022170306, 0.9462629050521761)

In [112]:
np.mean(heartRed_share_cor), np.mean(heartAll_share_cor)

(0.945840444409698, 0.9523935354593236)

In [32]:
kv.vocab

{'的': <gensim.models.keyedvectors.Vocab at 0x7fe085fc0828>,
 '年': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc18>,
 '月': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc50>,
 '日': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dc88>,
 '在': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dcc0>,
 '是': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dcf8>,
 '和': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dd30>,
 '了': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dd68>,
 '村': <gensim.models.keyedvectors.Vocab at 0x7fe06a48dda0>,
 '人': <gensim.models.keyedvectors.Vocab at 0x7fe06a48ddd8>,
 '中': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de10>,
 '为': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de48>,
 '為': <gensim.models.keyedvectors.Vocab at 0x7fe06a48de80>,
 '被': <gensim.models.keyedvectors.Vocab at 0x7fe06a48deb8>,
 '斯': <gensim.models.keyedvectors.Vocab at 0x7fe06a48def0>,
 '有': <gensim.models.keyedvectors.Vocab at 0x7fe06a48df28>,
 '及': <gensim.models.keyedvectors.Vocab 